In [10]:
import networkx as nx
from collections import defaultdict
import csv
from datetime import timedelta
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect
import numpy as np
from collections import defaultdict
import random
import gurobipy as gp 
from gurobipy import GRB
from models import simple_mpc, mpc_duration_constr, lazy, column_generation
from helper import fetch_data, create_duty_graph, node_legal

In [ ]:
def create_duty_graph(services):
    '''
    Creates a directed graph of services, with source and sink nodes at -2 and -1 respectively.

    Arguments:
        services - list of Service objects

    Returns:
        A directed graph G
    '''
    G = nx.DiGraph()
    G.add_node(-2)  # start node
    G.add_node(-1)  # end node

    # Add nodes for each service
    for service in services:
        G.add_node(service.serv_num)

    # Connect services based on legal connections
    for i, service1 in enumerate(services):
        for j, service2 in enumerate(services):
            if i != j and node_legal(service1, service2):
                G.add_edge(service1.serv_num, service2.serv_num,
                           weight=service1.serv_dur)

    # Connect start and end nodes
    for service in services:
        G.add_edge(-2, service.serv_num, weight=0)  # From start node
        G.add_edge(service.serv_num, -1, weight=service.serv_dur)  # To end node

    return G

In [3]:
services, service_dict = fetch_data('./StepBackServices.csv', partial=True, rakes = 1)       # 46 maximum number of rakes
graph = create_duty_graph(services)
print(graph)

DiGraph with 23 nodes and 66 edges


In [ ]:
def initial_restricted_master(G):
    model = gp.Model("Restricted_Master")
    model.setParam('OutputFlag', 0)  # Suppress output

    # Decision variables for edges
    edge_vars = {}
    for u, v in G.edges():
        edge_vars[(u, v)] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"flow_{u}_{v}")

    # Objective function: minimize total duration (change to suit your objective)
    model.setObjective(gp.quicksum(G[u][v]["weight"] * edge_vars[(u, v)] for u, v in G.edges()), GRB.MINIMIZE)

    # Flow balance constraints
    for node in G.nodes():
        if node == -2:  # Start node
            model.addConstr(gp.quicksum(edge_vars[(-2, j)] for j in G.successors(-2)) == 1)
        elif node == -1:  # End node
            model.addConstr(gp.quicksum(edge_vars[(i, -1)] for i in G.predecessors(-1)) == 1)
        else:  # Intermediate nodes
            in_flow = gp.quicksum(edge_vars[(i, node)] for i in G.predecessors(node))
            out_flow = gp.quicksum(edge_vars[(node, j)] for j in G.successors(node))
            model.addConstr(in_flow == out_flow)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        edge_flows = {edge: edge_vars[edge].X for edge in G.edges()}
        dual_values = {node: constr.Pi for node, constr in zip(G.nodes(), model.getConstrs())}
        return model.ObjVal, edge_flows, dual_values
    else:
        raise RuntimeError("No optimal solution found for the initial restricted master.")

Primal Solution (Fractional Flow Values):
Edge (336, 1): Flow Value = 0.00
Edge (336, -1): Flow Value = 0.00
Edge (1, 702): Flow Value = 0.00
Edge (1, 23): Flow Value = 0.00
Edge (1, -1): Flow Value = 0.00
Edge (702, -1): Flow Value = 0.00
Edge (23, 366): Flow Value = 0.00
Edge (23, 63): Flow Value = 0.00
Edge (23, -1): Flow Value = 0.00
Edge (366, 63): Flow Value = 0.00
Edge (366, -1): Flow Value = 0.00
Edge (63, 741): Flow Value = 0.00
Edge (63, 102): Flow Value = 0.00
Edge (63, -1): Flow Value = 0.00
Edge (741, -1): Flow Value = 0.00
Edge (102, 405): Flow Value = 0.00
Edge (102, 135): Flow Value = 0.00
Edge (102, -1): Flow Value = 0.00
Edge (405, 135): Flow Value = 0.00
Edge (405, -1): Flow Value = 0.00
Edge (135, 777): Flow Value = 0.00
Edge (135, 168): Flow Value = 0.00
Edge (135, -1): Flow Value = 0.00
Edge (777, -1): Flow Value = 0.00
Edge (168, 438): Flow Value = 0.00
Edge (168, 202): Flow Value = 0.00
Edge (168, -1): Flow Value = 0.00
Edge (438, 202): Flow Value = 0.00
Edge (4

In [11]:
def pricing_problem(G, dual_values):
    """
    Find a negative reduced-cost path using the Bellman-Ford algorithm.
    """
    for u, v in G.edges():
        dual_u = dual_values.get(u, 0)
        reduced_cost = G[u][v]["weight"] - dual_u
        if reduced_cost < 0:  # Negative reduced cost
            return (u, v)

    return None

In [12]:
def column_generation(services):
    G = create_duty_graph(services)

    # Step 1: Solve Initial Restricted Master
    initial_obj, edge_flows, dual_values = initial_restricted_master(G)
    print(f"Initial Objective Value: {initial_obj}")

    while True:
        # Step 2: Solve Pricing Problem
        new_edge = pricing_problem(G, dual_values)

        if new_edge is None:  # No negative reduced-cost path found
            break

        # Add new edge to the graph with reduced cost and resolve
        G.add_edge(new_edge[0], new_edge[1], weight=G[new_edge[0]][new_edge[1]]["weight"])
        initial_obj, edge_flows, dual_values = initial_restricted_master(G)
        print(f"Updated Objective Value: {initial_obj}")

    return G, edge_flows

In [8]:
def compute_flow_values(G, primal_solution):
    """
    Compute and store flow values for each edge in the graph G.
    
    G: NetworkX DiGraph representing the network of duties.
    primal_solution: Dictionary with keys as (u, v) edge tuples and 
                     values as the fractional flow from the LP relaxation.
    """
    # Initialize flow values
    nx.set_edge_attributes(G, 0, "flow")  # Set default flow to 0

    for (u, v), flow_value in primal_solution.items():
        if G.has_edge(u, v):
            G[u][v]["flow"] = flow_value  # Assign the flow value to the edge

    print("Flow values have been updated on the graph.")
    return G

G = compute_flow_values(graph, primal_solution)

# Check the flow values stored in the graph
for u, v, data in G.edges(data=True):
    print(f"Edge ({u} -> {v}): Flow Value = {data['flow']}")


Flow values have been updated on the graph.
Edge (336 -> 1): Flow Value = 0.0
Edge (336 -> -1): Flow Value = 0.0
Edge (1 -> 702): Flow Value = 0.0
Edge (1 -> 23): Flow Value = 0.0
Edge (1 -> -1): Flow Value = 0.0
Edge (702 -> -1): Flow Value = 0.0
Edge (23 -> 366): Flow Value = 0.0
Edge (23 -> 63): Flow Value = 0.0
Edge (23 -> -1): Flow Value = 0.0
Edge (366 -> 63): Flow Value = 0.0
Edge (366 -> -1): Flow Value = 0.0
Edge (63 -> 741): Flow Value = 0.0
Edge (63 -> 102): Flow Value = 0.0
Edge (63 -> -1): Flow Value = 0.0
Edge (741 -> -1): Flow Value = 0.0
Edge (102 -> 405): Flow Value = 0.0
Edge (102 -> 135): Flow Value = 0.0
Edge (102 -> -1): Flow Value = 0.0
Edge (405 -> 135): Flow Value = 0.0
Edge (405 -> -1): Flow Value = 0.0
Edge (135 -> 777): Flow Value = 0.0
Edge (135 -> 168): Flow Value = 0.0
Edge (135 -> -1): Flow Value = 0.0
Edge (777 -> -1): Flow Value = 0.0
Edge (168 -> 438): Flow Value = 0.0
Edge (168 -> 202): Flow Value = 0.0
Edge (168 -> -1): Flow Value = 0.0
Edge (438 -> 

In [9]:
def variable_rounding_upper_bound(G, source, sink):
    """
    Implement the VR-UB approach using computed flow values.
    
    G: NetworkX DiGraph with 'flow' attribute on each edge.
    source: Source node of the graph.
    sink: Sink node of the graph.
    """
    path = [source]
    current_node = source
    total_cost = 0

    while current_node != sink:
        # Find the outgoing edge with maximum non-zero flow
        next_edge = max(
            (edge for edge in G.out_edges(current_node, data=True) if edge[2].get("flow", 0) > 0),
            key=lambda x: x[2]["flow"],
            default=None
        )

        if next_edge is None:
            raise ValueError("No valid path to the sink node. Check feasibility or primal solution.")

        next_node = next_edge[1]
        total_cost += next_edge[2].get("flow", 0)  # Adjust based on objective function weight if needed

        path.append(next_node)
        current_node = next_node

    return path, total_cost

# Running the VR-UB function
source, sink = -2, -1
path, cost = variable_rounding_upper_bound(G, source, sink)
print(f"Rounded Path: {path}")
print(f"Total Cost: {cost}")


Rounded Path: [-2, 475, -1]
Total Cost: 2.0
